In [ ]:
!pip install transformers accelerate huggingface_hub

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained(
        "jinaai/starcoder-1b-textbook", device_map='auto'
    )

tokenizer = AutoTokenizer.from_pretrained("jinaai/starcoder-1b-textbook")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

In [ ]:
prompt = '''
generate a python function that given a positive integer, it returns its roman numeral equivalent as a string,

def int_to_mini_roman
'''

inputs = tokenizer(prompt.rstrip(), return_tensors="pt").to("cuda")

generation_output = model.generate(
    **inputs,
    max_new_tokens=256,
    eos_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
)

s = generation_output.sequences[0]
output = tokenizer.decode(s, skip_special_tokens=True)

print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



generate a python function that given a positive integer, it returns its roman numeral equivalent as a string,

def int_to_mini_romann(num):
    roman_numerals = {
        1000: 'M',
        900: 'CM',
        500: 'D',
        400: 'CD',
        100: 'C',
        90: 'XC',
        50: 'L',
        40: 'XL',
        10: 'X',
        9: 'IX',
        5: 'V',
        4: 'IV',
        1: 'I'
    }
    
    roman_numeral = ""
    for i in range(1, num + 1):
        roman_numeral += roman_numerals[i]
    
    return roman_numeral


# Exercise:
# Complete the code below to find the sum of the digits in the given integer.
# Use the int_to_mini_numeral function to convert the integer to a string.
# Finally, print the sum of the digits.

integer = 123456789

# Convert the integer to a string
integer_str = int_to_mini_numeral(integer)

# Calculate the sum of the digits


In [ ]:
!pip install transformers accelerate huggingface_hub datasets
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# "textbooks are all you need" case study

In [25]:
from datasets import Dataset

# Convert the list of dictionaries to a dictionary of lists
synthetic_code_snippets = {
    "text": [
        "def add(a, b):\n    return a + b",
        "def subtract(a, b):\n    return a - b",
        "def multiply(a, b):\n    return a * b",
        "def divide(a, b):\n    if b != 0:\n        return a / b\n    else:\n        return None"
    ]
}

# Create a Dataset object from the corrected format
code_dataset = Dataset.from_dict(synthetic_code_snippets)

# Print the dataset length to confirm creation
print(f"Length of synthetic dataset: {len(code_dataset)}")


Length of synthetic dataset: 4


In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer

# Load the pretrained Starcoder model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("jinaai/starcoder-1b-textbook")
model = AutoModelForCausalLM.from_pretrained("jinaai/starcoder-1b-textbook").to("cuda")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize the entire synthetic dataset
tokenized_code_dataset = code_dataset.map(tokenize_function, batched=True)

# Check an example to verify the output
print(tokenized_code_dataset[0])


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'text': 'def add(a, b):\n    return a + b', 'input_ids': [589, 1015, 26, 83, 30, 323, 711, 284, 442, 312, 474, 323], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [29]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Evaluate after every epoch
    learning_rate=5e-5,           # Initial learning rate
    num_train_epochs=3,           # Number of epochs
    weight_decay=0.01,            # Regularization parameter
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    save_total_limit=2,           # Save only the most recent checkpoints
    fp16=True,                    # Enable mixed precision for faster training
)

print("Training arguments set up successfully.")


Training arguments set up successfully.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Create a Trainer object for training the model
trainer = Trainer(
    model=model,                  # Pre-trained model
    args=training_args,           # Training arguments
    train_dataset=tokenized_code_dataset,  # Training dataset
)

# Train the model
trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
# Generate predictions from the model after training
prompt = "def fibonacci(n: int) -> int:\n    \"\"\"Calculate nth Fibonacci number.\"\"\"\n"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate continuation of the input prompt
generation_output = model.generate(
    inputs["input_ids"],
    max_new_tokens=50,
    temperature=0.7,  # Controls randomness of generation
    eos_token_id=tokenizer.eos_token_id
)

# Decode and print the generated output
output = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(output)
